In [20]:
import numpy as np
import csv
import astropy.coordinates as coord
import astropy.units as u
from astropy.coordinates import Galactic
from astropy.coordinates import SkyCoord
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy.coordinates import AltAz
import matplotlib.pyplot as plt
from matplotlib.projections import get_projection_names
from astropy.coordinates import ICRS
import math
import pandas as pd
#################################################################################################
def mjdTOgps(value):
    """This function is only for values"""
    gps_origin = pd.Timestamp('1980-01-06')
    MJD_origin = pd.Timestamp('1858-11-17')
    # Calculate time difference from GPS origin
    d= (value * 86400) - gps_origin.timestamp()+MJD_origin.timestamp()
    return d
#################################################################################################
### HAWC information
HAWC= EarthLocation(lat='18.994722', lon='-97.3085', height=4100*u.m)
fov=25
### Data of GRBs
FILE='ShortGRBs2014_2022_3.5_-10.0_50.02.dat'
FILE_RESULTS='GRBsAngleFromZenith2.dat'
# FILE="ShortGRBs2014_2022_3.5_-10.0_50.0.dat"
# FILE_RESULTS='GRBsAngleFromZenith.dat'
name= np.genfromtxt(FILE, dtype=str)[:,0]
results=np.genfromtxt(FILE)

RA, DEC, Trigg= results[:,1], results[:,2], results[:,4]
      
### Closest Zenith of GRBs
f = open(FILE_RESULTS,'w')
f.write('GRB'+ '\t' +'AngleFromZenith'+'\t'+ 'MinutesSinceTrigger'+'\t'+'Time1FOV'+'\t'+'Time2FOV'+'\n')
print('Closest Zenith to GRB')
for idx, grb in(enumerate(name)):
    Time0=Time(Trigg[idx], format='mjd')
    gpsT=mjdTOgps(Trigg[idx])
    ragrb = coord.Angle(RA[idx]*u.degree)
    ragrb = ragrb.wrap_at(180*u.degree)
#    decgrb = coord.Angle(DEC[idx]*u.degree)
    ZenAng=[]
    Zen=[]
    Indices=[]
    fig, ax = plt.subplots(figsize=(8, 6))
    ax = plt.subplot(1,1,1)
    IndPlot=[]
    ZenPlot=[]
    print(grb)
#    for i in range(0,48):
#        Time01=Time0+i*0.5*u.h
    for i in range(0,1440):
        Time01=Time0+i*u.min
        SkyZen01= SkyCoord(az=0., alt=90, unit="deg", frame=AltAz(obstime=Time01, location=HAWC))
        RAzen01=SkyZen01.icrs.ra.deg  
        raZen = coord.Angle(RAzen01*u.degree)
        raZen = raZen.wrap_at(180*u.degree)        
        DECzen01=SkyZen01.icrs.dec.deg
 #       print(i, raZen.deg, ragrb.deg)
  
### In angle
        ra1rad=np.deg2rad(ragrb.deg)
        ra2rad=np.deg2rad(raZen.deg)
        dec1rad=np.deg2rad(DEC[idx])
        dec2rad=np.deg2rad(DECzen01)
#        AA=math.cos(ra1rad)
        NumZ=(math.cos(dec2rad)**2.*math.sin(ra2rad-ra1rad)**2.)+(math.cos(dec1rad)*math.sin(dec2rad)-math.sin(dec1rad)*math.cos(dec2rad)*math.cos(ra2rad-ra1rad))**2.
        NumZenF=np.sqrt(NumZ)
        DenZenF=math.sin(dec1rad)*math.sin(dec2rad)+math.cos(dec1rad)*math.cos(dec2rad)*math.cos(ra2rad-ra1rad)
        AnFromZen=math.degrees(math.atan(NumZenF/DenZenF))
        ZenAng=np.append(ZenAng, AnFromZen)
### For validation
        if ragrb.deg > 0 and raZen.deg <0:
            RAZen=(180-ragrb.deg)+(180-abs(raZen.deg))
            if RAZen > 180:
                RAZen= ragrb.deg-raZen.deg
        if ragrb.deg > 0 and raZen.deg >0:
            RAZen=ragrb.deg-raZen.deg
        if ragrb.deg < 0 and raZen.deg < 0:
            RAZen= raZen.deg-ragrb.deg
        if ragrb.deg <0 and raZen.deg >0:
            RAZen=(180-raZen.deg)+(180 -abs(ragrb.deg))
            if RAZen > 180:
 #               print('pasa 180')
                RAZen=raZen.deg-ragrb.deg
 #           ragrb.deg=ragrb.deg
#        print(RAZen)
        zenith=np.sqrt((RAZen)**2+(DECzen01-DEC[idx])**2)
        IndPlot=np.append(IndPlot,i)
        ZenPlot=np.append(ZenPlot,zenith)
 #       if zenith < fov:
        Zen=np.append(Zen, zenith)
        Indices=np.append(Indices, i)
    index_min = np.argmin(Zen)
    Angtiempo1=Zen[Zen<45][0]
    Angtiempo2=max(Zen[Zen<45])
    tiempo1=np.where(Zen==Angtiempo1)[0][0]*60+gpsT
    tiempo2=np.where(Zen==Angtiempo2)[0][0]*60+gpsT
    f.write(str(grb)+'\t'+str(min(Zen))+'\t'+str(Indices[index_min]*60+gpsT)+'\t'+str(tiempo1)+'\t'+str(tiempo2)+'\n')
    print(grb, min(Zen), Indices[index_min],Indices[index_min]*60+gpsT,tiempo1,tiempo2)
    ax.scatter(IndPlot, ZenAng, color='black')
    ax.set_ylim(0, 90)
    ax.set_xlabel('Time since trigger (min)', fontsize=16)
    ax.set_ylabel('Angle from Zenith HAWC', fontsize=16)
    ax.set_title(grb, fontsize=18)
    plt.gca().invert_yaxis()
    plt.savefig('ZenithPlots/'+grb+'ZenitAng.png', format='png', dpi=300)
    plt.close()
f.close()

Closest Zenith to GRB
GRB141206254
GRB141206254 16.506321545534213 1004.0 1101941391.7100353 1101931371.7100353 1101931371.7100353
GRB141208632
GRB141208632 7.531405011676294 606.0 1102122958.319395 1102112398.319395 1102133578.319395
GRB141205337
GRB141205337 18.874427141696316 1416.0 1101886877.4907675 1101801917.4907675 1101810497.4907675


In [21]:
Angtiempo2=max(Zen[Zen<45][1::])
tiempo2=np.where(Zen==Angtiempo2)[0][0]*60+gpsT
Angtiempo2,(tiempo2-gpsT)/(60)
def convert_gps_to_reduced_julian_SIMPLE(value):
    # Define GPS time origin (January 6, 1980, midnight UTC)
    gps_origin = pd.Timestamp('1980-01-06')
    MJD_origin = pd.Timestamp('1858-11-17')
    # Calculate time difference from GPS origin
    d = (value + gps_origin.timestamp()-MJD_origin.timestamp())/86400
    return d
tiempo1, tiempo2,convert_gps_to_reduced_julian_SIMPLE(tiempo1),convert_gps_to_reduced_julian_SIMPLE(tiempo2)

(1101801917.4907675, 1101810497.4907675, 56996.337007995, 56996.43631355055)

In [26]:
Angtiempo2=max(Zen[Zen<45])
tiempo2=np.where(Zen==Angtiempo2)[0][0]*60+gpsT
tiempo2
convert_gps_to_reduced_julian_SIMPLE(1101810497.4907675)
convert_gps_to_reduced_julian_SIMPLE(1101801917.4907675)
Zen[0:720]

array([ 19.51253578,  19.57764083,  19.64573042,  19.71677367,
        19.7907388 ,  19.86759321,  19.94730353,  20.0298357 ,
        20.11515501,  20.20322618,  20.29401342,  20.38748047,
        20.48359068,  20.58230705,  20.68359229,  20.78740889,
        20.89371913,  21.00248519,  21.11366913,  21.22723299,
        21.34313882,  21.46134868,  21.58182476,  21.70452934,
        21.82942487,  21.95647399,  22.08563955,  22.21688468,
        22.35017277,  22.4854675 ,  22.6227329 ,  22.76193335,
        22.9030336 ,  23.04599876,  23.19079438,  23.33738641,
        23.48574123,  23.63582568,  23.78760705,  23.94105308,
        24.09613199,  24.25281249,  24.41106376,  24.57085547,
        24.73215779,  24.89494138,  25.0591774 ,  25.22483749,
        25.3918938 ,  25.56031899,  25.73008619,  25.90116904,
        26.07354166,  26.24717865,  26.42205512,  26.59814664,
        26.77542925,  26.95387947,  27.13347429,  27.31419115,
        27.49600794,  27.678903  ,  27.86285513,  28.04